# Building, tracking and plotting in a beamline

This notebook illustrates the following points:
 - Building a beamline with Zgoubidoo, using a variety of elements
 - Tracking using a few test particles
 - Plotting the beamline and the tracks in a laboratory reference frame

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import zgoubidoo
from zgoubidoo.commands import *
from zgoubidoo import ureg as _

In [ ]:
XPAS = 5 * _.mm
zi = zgoubidoo.Input('test-line')
zi += Objet2('BUNCH', BORO=2149 * _.kilogauss * _.cm).add(
    np.array(
        [
            [0, 0, 0, 0, 1.0],
            [0, 0, 0, 0, 1.2],
        ]
    )
)   
zi += Proton()
zi += Ymy('INITIAL_YMY')
zi += Dipole('DIPOLE_1', RM=200 * _.cm, AT=30 * _.degree, B0=10.745 * _.kilogauss, N=0,
             IL=2,
             ACENT=30 * _.degree / 2,
             IORDRE=25,
             KPOS=2,
             XPAS=0.1 * _.centimeter,
             OMEGA_E=(30 * _.degree / 2),
             OMEGA_S=(-30 * _.degree / 2),
             RE=200 * _.centimeter,
             TE=0 * _.radian,
             RS=200 * _.centimeter,
             TS=0 * _.radian)
zi += Quadrupole('QUAD_1',
                 XL=15 * _.cm,
                 B0=0.0001 * _.tesla,
                 XCE=0.0 * _.centimeter,
                 YCE=0.0 * _.centimeter,
                 ALE=0. * _.degree,
                 XPAS=XPAS,
                 KPOS=2,
                 )
zi += Dipole('DIPOLE_2', RM=200 * _.cm, AT=45 * _.degree, B0=10.745 * _.kilogauss, N=0,
             IL=2,
             ACENT=45 * _.degree / 2,
             IORDRE=25,
             KPOS=2,
             XPAS=0.1 * _.centimeter,
             OMEGA_E=(45 * _.degree / 2),
             OMEGA_S=(-45 * _.degree / 2),
             RE=200 * _.centimeter,
             TE=0 * _.radian,
             RS=200 * _.centimeter,
             TS=0 * _.radian)
zi += Ymy('INNER_YMY')
zi += Quadrupole('QUAD_1',
                 XL=15 * _.cm,
                 B0=0.1 * _.tesla,
                 XCE=0.0 * _.centimeter,
                 YCE=-0.0 * _.centimeter,
                 ALE=0. * _.degree,
                 XPAS=XPAS,
                 KPOS=2,
                 )
zi += ChangeRef(
    TRANSFORMATIONS=[('YS', 50 * _.cm), ('ZR', -15 * _.degree)]
)
zi += Dipole('DIPOLE_3', RM=200 * _.cm, 
             AT=30 * _.degree, 
             B0=20.745 * _.kilogauss, N=0,
             IL=2,
             ACENT=30 * _.degree / 2,
             IORDRE=25,
             KPOS=2,
             XPAS=0.1 * _.centimeter,
             OMEGA_E=(30 * _.degree / 2),
             OMEGA_S=(-30 * _.degree / 2),
             RE=200 * _.centimeter,
             TE=0 * _.radian,
             RS=200 * _.centimeter,
             TS=0 * _.radian)
zi += Ymy()
zi += Quadrupole('QUAD_2',
                 XL=50 * _.cm,
                 B0=0.00001 * _.tesla,
                 XCE=0.0 * _.centimeter,
                 YCE=0.0 * _.centimeter,
                 ALE=0. * _.degree,
                 XPAS=XPAS,
                 KPOS=2,
                 )
zi += Bend('BEND_1',
           POLE_WIDTH=60 * _.cm,
           XL=100 * _.cm,
           B1=0.1 * _.tesla,
           KPOS=1,
           X_E=1 * _.cm,
           LAM_E=5 * _.cm,
           X_S=1 * _.cm,
           LAM_S=5 * _.cm,
           LENGTH_IS_ARC_LENGTH=False,
           )
zi += Decapole('DECA_1',
               XL=50 * _.cm,
               B0=0.0001 * _.gauss,
               R0=20 * _.cm,
               XCE=0.0 * _.centimeter,
               YCE=-0.0 * _.centimeter,
               ALE=0. * _.degree,
               XPAS=XPAS,
               KPOS=1,
               )
zi += Dipole('DIPOLE_4', RM=200 * _.cm, AT=30 * _.degree, B0=10.745 * _.kilogauss, N=0,
             IL=2,
             ACENT=30 * _.degree / 2,
             IORDRE=25,
             KPOS=2,
             XPAS=0.1 * _.centimeter,
             OMEGA_E=(30 * _.degree / 2),
             OMEGA_S=(-30 * _.degree / 2),
             R1_E=1e9 * _.centimeter,
             U1_E=1e9 * _.centimeter,
             U2_E=1e9 * _.centimeter,
             R2_E=1e9 * _.centimeter,
             RE=200 * _.centimeter,
             TE=0 * _.radian,
             RS=200 * _.centimeter,
             TS=0 * _.radian
             )
z = zgoubidoo.Zgoubi()
zr = z(zi).collect()
# print('\n'.join(out['result']))

In [ ]:
zi.survey();

In [ ]:
artist = zgoubidoo.vis.ZgoubidooPlotlyArtist()
artist.plot_beamline(beamline=zi)

particle1 = zr.tracks_global[zr.tracks_global['IT'] == 1]
particle2 = zr.tracks_global[zr.tracks_global['IT'] == 2]
artist.scatter(x=particle1['XG'], 
               y=particle1['YG'],)

artist.scatter(x=particle2['XG'], 
               y=particle2['YG'],)
artist.render()

In [ ]:
# line = zgoubidoo.survey(beamline=zi, reference_frame=Frame())
# fig = plt.figure(figsize=(12, 8))
# ax = fig.add_subplot(111)

# zmpl = zgoubidoo.plotting.ZgoubiMpl(ax, with_frames=True, with_boxes=True, with_centers=True, with_drifts=True, tracks_color='b')
# #tracks = zgoubidoo.read_plt_file()
# zgoubidoo.plotting.plot(beamline=line, 
#                         artist=zmpl, 
#                         tracks=None, 
#                         )
# ax.set_aspect('equal', 'datalim')